<a href="https://colab.research.google.com/github/dantae74/machine-learning/blob/main/stock-etc/99-03-rnn-lot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

모두를 위한 머신러닝에서 가져왔습니다.
# RNN-lot


In [ ]:
# !pip install tensorflow-gpu==2.0.0
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from io import BytesIO
import requests

# 마지막 회차번호

In [ ]:
# 마지막 회차 번호
noEnd ='970' 

# url
# url = 'https://dhlottery.co.kr/gameResult.do?method=allWinExel&gubun=byWin&nowPage=&drwNoStart=1&drwNoEnd={}'.format(noEnd)

url = f'https://dhlottery.co.kr/gameResult.do?method=allWinExel&gubun=byWin&nowPage=&drwNoStart=1&drwNoEnd={noEnd}'

# user-agent
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"

In [ ]:
result = requests.get(url, headers={"user-agent":user_agent})

In [ ]:
result

In [ ]:
f = BytesIO(result.content)
# df = pd.read_excel(f,sheet_name='excel',skiprows=3)
html_result = pd.read_html(f)

In [ ]:
df_raw = pd.DataFrame(html_result[1])
# df = df.drop([0])
df_raw.head()

In [ ]:
# columns = [col for i, col in enumerate(df.iloc[0])]
# df.columns = columns
# columns

# column name 설정
col_names = [fn+sn if fn != sn else fn for fn, sn in zip(df_raw.iloc[0], df_raw.iloc[1])]  
df_raw.columns = col_names

# 0, 1행 삭제
df_lot = df_raw.drop([0,1], axis=0)

# 컬럼 형변환
df_lot = df_lot.astype({'회차':'uint', '당첨번호1':'uint', '당첨번호2':'uint', '당첨번호3':'uint', '당첨번호4':'uint', '당첨번호5':'uint', '당첨번호6':'uint', '당첨번호보너스':'uint'}, copy=True)

# 당첨금액 형변환
df_lot['1등당첨금액'] = df_lot['1등당첨금액'].str.replace(",|원",'',regex=True).astype('uint')
df_lot['2등당첨금액'] = df_lot['2등당첨금액'].str.replace(",|원",'',regex=True).astype('uint')
df_lot['3등당첨금액'] = df_lot['3등당첨금액'].str.replace(",|원",'',regex=True).astype('uint')
df_lot['4등당첨금액'] = df_lot['4등당첨금액'].str.replace(",|원",'',regex=True).astype('uint')
df_lot['5등당첨금액'] = df_lot['5등당첨금액'].str.replace(",|원",'',regex=True).astype('uint')

# 회차기준 오름차준 정렬
df_lot = df_lot.sort_values('회차', ascending=True)

# 회차필드 index로 설정
df_lot.set_index('회차', inplace=True)

# 필요없는 컬럼 삭제
df_lot.drop(['년도', '추첨일', '1등당첨자수', '2등당첨자수', '3등당첨자수', '4등당첨자수', '5등당첨자수'], axis=1, inplace=True)

df_lot.head(5)

In [ ]:
df_lot.info()

In [ ]:
import numpy as np

# 당첨번호를 원핫인코딩벡터로 변환
def lotto2onehot(numbers):
    onehot = np.zeros(45)
    for number in numbers:
        onehot[int(number)-1] = 1 #로또번호에 해당하는 인덱스에 1로 설정 
    return onehot.tolist()

# 원핫인코딩벡터를 당첨번호로 변환
def onehot2lotto(onehots):
    numbers = [i+1 for i, val in enumerate(onehots) if val == 1]
    return numbers

In [ ]:
# 함수 동작 확인

lotto = [1,3,5,7,9,11]
onehot_lotto = lotto2onehot(lotto)
print(onehot_lotto)
onehot2lotto(onehot_lotto)

In [ ]:
# 당첨번호 추출
bonus_number = df_lot['당첨번호보너스'].to_numpy()
lotto_numbers = df_lot[['당첨번호1','당첨번호2','당첨번호3', '당첨번호4', '당첨번호5', '당첨번호6']].to_numpy()
lotto_numbers

In [ ]:
# 당첨번호 onehot 인코딩
onehot_lotto_numbers = list(map(lotto2onehot, lotto_numbers))

row_count = len(onehot_lotto_numbers)

x_samples = onehot_lotto_numbers[0:row_count-1]
y_samples = onehot_lotto_numbers[1:row_count]

In [ ]:
np.shape(onehot_lotto_numbers)

In [ ]:
# train, valid, test set으로 분리
train_idx = (0, 800)
valid_idx = (801, 900)
test_idx = (901, len(onehot_lotto_numbers))

print(f'train: {train_idx}, valid: {valid_idx}, test: {test_idx}')

In [ ]:
# LSTM 모델 생성

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models

batch_size = 32
lotto_size = 45
embedding_dim = 256
rnn_units = 256


# 모델을 정의합니다.
model = keras.Sequential([
    # keras.layers.Embedding(lotto_size, embedding_dim, batch_input_shape=[batch_size, None]),                      
    keras.layers.LSTM(rnn_units, return_sequences=False, stateful=True, recurrent_initializer='glorot_uniform'),
    keras.layers.Dense(45, activation='sigmoid')
])

# 모델을 컴파일합니다.
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
import os
import time

# 체크포인트가 저장될 디렉토리
checkpoint_dir = './training_checkpoints'
# 체크포인트 파일 이름
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
EPOCHS=10

x_train = x_samples[train_idx[0]:train_idx[1]]
y_train = y_samples[train_idx[0]:train_idx[1]]

x_val = x_samples[valid_idx[0]:valid_idx[1]]
y_val = y_samples[valid_idx[0]:valid_idx[1]]

x_test = x_samples[test_idx[0]:test_idx[1]]
y_test = y_samples[test_idx[0]:test_idx[1]]

# history = model.fit(x_train, y_train, epochs=EPOCHS, callbacks=[checkpoint_callback])

history = model.fit(x_train, y_train, epochs=EPOCHS, validation_data=(x_val, y_val), callbacks=[checkpoint_callback])

In [ ]:
np.shape(x_val)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

fig, loss_ax = plt.subplots()

acc_ax = loss_ax.twinx()

loss_ax.plot(history.history['loss'], 'y', label='train loss')
# loss_ax.plot(history.history['val_loss'], 'r', label='val loss')

acc_ax.plot(history.history['accuracy'], 'b', label='train acc')
# acc_ax.plot(history.history['val_acc'], 'g', label='val acc')

loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
acc_ax.set_ylabel('accuray')

loss_ax.legend(loc='upper left')
acc_ax.legend(loc='lower left')

plt.show()

In [ ]:
np.shape(x_val)

In [ ]:
del model